In [1]:
from ysdc_dataset_api.dataset import MotionPredictionDataset

/home/cds-k/anaconda3/envs/traj_sdc_dataset/lib/python3.6/site-packages/ysdc_dataset_api/utils/transform.py:90: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, C))
  res = transform @ ph
/home/cds-k/anaconda3/envs/traj_sdc_dataset/lib/python3.6/site-packages/numba/core/typing/npydecl.py:937: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, C))
  warnings.warn(NumbaPerformanceWarning(msg))


In [2]:
# Load protobufs for training dataset
dataset_path = '/media/cds-k/Data_2/canonical-trn-dev-data/data/train_pb/'

# Path to file with training scene tags
scene_tags_fpath = '/media/cds-k/Data_2/canonical-trn-dev-data/data/train_tags.txt'

In [3]:
import json
with open('/home/cds-k/Desktop/motion_prediction/Trajectron-plus-plus/experiments/nuScenes/train_configs/config_ph_25_maxhl_24_minhl_24.json', 'r', encoding='utf-8') as conf_json:
    hyperparams = json.load(conf_json)

In [4]:
import numpy as np

standardization = {
    'PEDESTRIAN': {
        'position': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1}
        },
        'velocity': {
            'x': {'mean': 0, 'std': 2},
            'y': {'mean': 0, 'std': 2}
        },
        'acceleration': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1}
        }
    },
    'VEHICLE': {
        'position': {
            'x': {'mean': 0, 'std': 80},
            'y': {'mean': 0, 'std': 80}
        },
        'velocity': {
            'x': {'mean': 0, 'std': 15},
            'y': {'mean': 0, 'std': 15},
            'norm': {'mean': 0, 'std': 15}
        },
        'acceleration': {
            'x': {'mean': 0, 'std': 4},
            'y': {'mean': 0, 'std': 4},
            'norm': {'mean': 0, 'std': 4}
        },
        'heading': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1},
            '°': {'mean': 0, 'std': np.pi},
            'd°': {'mean': 0, 'std': 1}
        }
    }
}

In [5]:
import sys
sys.path.append("../../../trajectron")
from environment import Environment, Scene, Node
from environment import derivative_of as derivative_of
env = Environment(node_type_list=['VEHICLE', 'PEDESTRIAN'], standardization=standardization)
attention_radius = dict()
attention_radius[(env.NodeType.PEDESTRIAN, env.NodeType.PEDESTRIAN)] = 10.0
attention_radius[(env.NodeType.PEDESTRIAN, env.NodeType.VEHICLE)] = 20.0
attention_radius[(env.NodeType.VEHICLE, env.NodeType.PEDESTRIAN)] = 20.0
attention_radius[(env.NodeType.VEHICLE, env.NodeType.VEHICLE)] = 30.0

env.attention_radius = attention_radius


In [6]:
env.NodeType[0]

VEHICLE

In [7]:
dataset = MotionPredictionDataset(
    dataset_path=dataset_path,
    scene_tags_fpath=scene_tags_fpath,
    feature_producer=None,
    transform_ground_truth_to_agent_frame=True,
    hyperparams=hyperparams,
    node_type=env.NodeType[0]
)

388406/388406 scenes fit the filter criteria.


In [8]:
dataset_iter = iter(dataset)

In [9]:
for i in range(1):
    data_item = next(dataset_iter)

KeyError: 'min_future_timesteps'